Author: Carlos Roberto de Melo

Date: 05/27/2020

Obj: Perform the modeling of spectrum based on pPXF method by Cappellari,and obtein the kinematics./Realizar o modelamento de espectros baeado no código pPXF de Cappellari, e obter a cinemática.

In [1]:
import glob
from os import path

from astropy.io import fits
from scipy import ndimage
import numpy as np

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util

import matplotlib.pyplot as plt

Adaptation of the Ppxf code to apply different spectra.
**x** and **y** are positions in px, which serves as counter and also index to open the spectrum files, which were saved in the format **y_px X x_px.fits**.

As discussed in the README, the .fits files containing the spectrum have three extensions. Note that pyton, by default, starts a count of zero.

It should be viewed as units of wavelength and flux. By default, pPXF works with wavelength in Angstrons, as we are using the MILES star library. In our case, as we normalize the spectrum, as units of intensity should not cause problems.



Adapatação do código Ppxf para se aplicar a diversos espectros.
x e y são as posições em px, de modo que servem de contador e também de index para abrir os arquivos de espectro, que foram salvos no formato **y_px X x_px.fits**.

Como discutido no README, os arquivos .fits que contém os espectros possuem três extensões. Note que o pyton, por padrão, começa a contagem do zero.

Deve-se antentar as unidades de comprimento de onda e fluxo. Por padrão, o ppxf trabalha com comprimento de onda em Angstrons pois estamos utilizando a biblioteca estelar do MILES. No nosso caso como vamos normalizar o espectro, as unidades de intensidade não devem gerar problemas.



In [2]:
ppxf_dir = '/home/carlos/Desktop/ppxf' #Directory of stellar library/Diretória da biblioteca estelar

stellar_lib = '/home/carlos/Downloads/Bibliotecas Estelares/BINTABLE'
stellar_G = '/home/carlos/Downloads/Bibliotecas Estelares/G'
Indo = glob.glob(stellar_lib + '/*.fits')
FWHM_tem = 1.35
chi_dof = []
cont = 0 
for y in range(20):
    for x in range(20):
        chi2_dof = 1
        print(chi2_dof)
    #READING THE SPECTRUM/LENDO UM ESPECTRO
        file = ('/home/carlos/Documents/GitHub/Master-Degree/ESO325/Dynamical Modelling/pPXF/Espectros/%dx%d.fits'%(y,x)) #Path and name of the spectrum/Caminho e nome do arquivo do espectro
        hdu = fits.open(file) #Opening the .fits file/Abrindo o arquivo.fits
        gal_lin = hdu[1].data #Read the data/Lendo os dados
        h1 = hdu[1].header    #Read the header/Lendo o header dos dados
        
    #READING THE VARIANCE/LENDO A VARIÂNCIA DO ESPECTRO
        var = hdu[2].data     #Read the variance/Lendo a variância
        h_var = hdu[2].header #Read the variance header/Lendo o header da variância
        
    #LOG-REBIN THE SPECTRUM AND VARIANCE/LOGARITMIZANDO O ESPECTRO E A VARIÂNCIA
        #SPECTRUM/ESPECTRO
        FWHM_gal = 2.71  #FWHM of telescope in Angstrons/FWHM do telescópio em Angstrons
        lambRange1 = h1['CRVAL1'] + np.array([0.,h1['CDELT1']*(h1['NAXIS1']-1)]) #Generating the wavelengths/Está linha gera os comprimentos de onda
                                #pois a variável gal_lin armazena em um array apenas as intensidades, mas não os comprimentos de onda
        
        #De-redshift
        z = 0.035 # Initial estimate of the galaxy redshift
        lambRange1 = lambRange1/(1+z) # Compute approximate restframe wavelength range
        FWHM_gal = FWHM_gal/(1+z)   # Adjust resolution in Angstrom
        #Depois do passo acima, a galáxia passa a estar no restframe, logo é preeciso setar
        z = 0.
        
        #Log-Rebin
        galaxy, logLam1, velscale = util.log_rebin(lambRange1, gal_lin) #Log-rebin the spectrum/Logaritimizando o espectro da galáxia
        median_galaxy = np.median(galaxy) #Mean of spectrum for normalization/Média do espectro, pois vamos normalizá-lo pela média
        galaxy = galaxy/median_galaxy        #Normalizing the spectrum/Normalizando o espectro pela média
        lam = np.exp(logLam1)                #Recovering the wavelength in Ansgstrons/Recuperando o comprimento de onda em unidades do SI
        
        #VARIANCE/VARIÂNCIA
        lamRange_var = h_var['CRVAL1'] + np.array([0.,h_var['CDELT1']*(h_var['NAXIS1']-1)])
        erro = np.sqrt(chi2_dof*var) #1-sigma error from variance/1-sigma erro da variância
        erro, loglam_var, velscale_var = util.log_rebin(lamRange_var, erro)
        noise = erro/median_galaxy         #Normalizing the error/Normalizando o erro        
        
     
    
    
    
    
    #READING ONE TEMPLATE FOR DETERMINE THE LENGHT/LENDO UM TEMPLATE PARA SABER O TAMANHO DO VETOR FINAL
        #The ideia is make a empty array to after complete with the informations about the templates of the stellar libary/
                    #A ideia é crirar um array vazio para que mais tarde seja preenchido com as informações de todos os templetes.
        
        
        velscale_ratio = 2
        hdu = fits.open(Indo[0])
        data = hdu[1].data
        ssp = data[0][1]
        h2 = hdu[0].header
        
    #Log_rebin TEMPLATE/LOGARITMIZANDO O TEMPLATE
        lamRange2 = np.array([data[0][0][0], data[0][0][len(data[0][0]) - 1]]) 
        sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
        templates = np.empty((sspNew.size, len(Indo)))  #Array com o tamanho necessário

    #FINAL FWHM FOR GALAXY AND TEMPLATE CONVOLUTION/CALCULANDO A FWHM FINAL PARA REALIZAR A CONVOLUÇÃO DO TEMPLATE E GALÁXIA
        FWHM_dif = np.sqrt(FWHM_gal**2 - FWHM_tem**2)
        sigma = FWHM_dif/2.355/0.4 #New sigma PSF/Novo sigma da PSF.
    
    #READING ALL TEMPLATES/LENDO TODOS OS TEMPLATES E JÁ LOGARITMIZANDO
        for j, file in enumerate(Indo):
            hdu = fits.open(file)
            data = hdu[1].data
            ssp = data[0][1]
            ssp = ndimage.gaussian_filter1d(ssp, sigma) #Realizando a convolução
            sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
            templates[:, j] = sspNew/np.median(sspNew)
    #ADICIONANDO UM SHIFT PARA COINCIDIR OS COMPRIMENTOS DE ONDA DO ESPECTRO E DO TEMPLATE
        #Artificial shift for the galaxy and templates wavelenght may coincide.    
        #Nem sempre os comprimentos de onda do templete e do espectro coincidem, por esse motivo é dado um 
        #shift artificial ao templete, mas que após o ajuste é retirado.
        c = 299792.458 #Speed of light/Velocidade da Luz
        dv = (np.mean(logLam2[:velscale_ratio]) - logLam1[0])*c #Shift
        goodPixels = util.determine_goodpixels(logLam1, lamRange2, z)  #Mask bad emission lines/Máscaras para linhas de emissão
        vel = c*np.log(1 + z)
        start = [vel, 200.] #Inicial guess/Chute inicial
    
    #START OF FIT/INÍCIO DO AJUSTE
       #For each spectrum the modeling is done. After the best solution is printed 
       #(velocity, velocity dispersion, another moments). Then, a table is write with the information about the
       #pixel position, velocity, velocity dispersion, chi²/DOF and 1-sigma error. The table is saved with name
       #'Output_Ppxf.txt'.  
        
       #Para cada espectro ele realiza o ajuste e printa abaixo o nome do px em que foi realizado o ajuste,
       #bem como as infomações do melhor ajuste (qui², Velocidade, Dispersão, Demais momentos). Então cria uma
       #tabela com as informações necessárias e a salva com o nome 'Output_Ppxf.txt'.
        pp = ppxf(templates, galaxy, noise, velscale, start,
                lam = lam, plot=True, moments=2,
                  degree=6,vsyst=dv, goodpixels=goodPixels, velscale_ratio=velscale_ratio)
    
        #For more informations about the method, visit the paper of Cappellari avaible in README
        #Para mais informações sobre o método, visite o paper de Cappellari disponível no README
        
        #Plot and save the best model/Plota e salva o melhor modelo
        
        pp.plot() #Plot the fit/Plota o fit
        plt.savefig('/home/carlos/Documents/GitHub/Master-Degree/ESO325/Dynamical Modelling/pPXF/Test2/%dx%d.png'%(y,x),format='png', dpi=1200)
        plt.clf()
        print(y,x)
        print('Vel :%f.\nDisp:%f.'%(pp.sol[0],pp.sol[1]))
        print('Best-fitting redshift z:', (0.035 + 1)*(1 + pp.sol[0]/c) - 1)
        error = pp.error*np.sqrt(pp.chi2) #1-sigma error/1-sigma erro
    #ESCREVENDO UMA TABELA COM OS DADOS DE Y_PX, X_PX,  VELOCIDADE(KM/S) E DISPERSÃO(KM/S)
        newrow = [np.int(y), np.int(x), pp.sol[0], pp.sol[1], pp.chi2, error[1]]
        if y == 0 and x == 0:
            table = np.array([np.int(y),np.int(x),pp.sol[0], pp.sol[1],pp.chi2, error[1]])
        else:
            table = np.vstack([table,newrow])
        print('\n####################\n')
        cont = cont + 1

1
 Best Fit:       Vel     sigma
 comp.  0:      -396       284
chi2/DOF: 1.665
method = capfit; Jac calls: 13; Func calls: 41; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
0 0
Vel :-396.022557.
Disp:284.251822.
Best-fitting redshift z: 0.03363277632288364

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -324       205
chi2/DOF: 1.965
method = capfit; Jac calls: 8; Func calls: 26; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
0 1
Vel :-324.413095.
Disp:205.105997.
Best-fitting redshift z: 0.03387999999715663

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -386       382
chi2/DOF: 1.662
method = capfit; Jac calls: 9; Func calls: 29; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
0 2
Vel :-385.924533.
Disp:381.854989.
Best-fitting redshift z: 0.03366763862310873

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -274       290
chi2/DOF: 1.455
method = c

 Best Fit:       Vel     sigma
 comp.  0:      -308       265
chi2/DOF: 1.41
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  13 / 1273
1 7
Vel :-308.065005.
Disp:264.771668.
Best-fitting redshift z: 0.03393643995462914

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -311       305
chi2/DOF: 1.291
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
1 8
Vel :-311.146923.
Disp:305.007694.
Best-fitting redshift z: 0.033925799975894844

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -339       272
chi2/DOF: 1.369
method = capfit; Jac calls: 8; Func calls: 29; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  11 / 1273
1 9
Vel :-338.659132.
Disp:272.407062.
Best-fitting redshift z: 0.03383081714638769

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -319       288
chi2/DOF: 1.394
method = ca

 Best Fit:       Vel     sigma
 comp.  0:      -355       290
chi2/DOF: 1.335
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
2 14
Vel :-355.004071.
Disp:289.739653.
Best-fitting redshift z: 0.03377438806720012

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -367       288
chi2/DOF: 1.446
method = capfit; Jac calls: 7; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  15 / 1273
2 15
Vel :-366.743320.
Disp:288.110080.
Best-fitting redshift z: 0.03373385962325326

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -308       281
chi2/DOF: 1.332
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
2 16
Vel :-307.920439.
Disp:280.647276.
Best-fitting redshift z: 0.033936939052563675

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -263       263
chi2/DOF: 1.351
method =

 Best Fit:       Vel     sigma
 comp.  0:      -299       306
chi2/DOF: 1.469
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
4 1
Vel :-299.086056.
Disp:305.792151.
Best-fitting redshift z: 0.0339674387751876

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -323       267
chi2/DOF: 1.132
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
4 2
Vel :-322.910385.
Disp:266.970467.
Best-fitting redshift z: 0.03388518793876849

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -359       309
chi2/DOF: 1.338
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
4 3
Vel :-359.412416.
Disp:308.849440.
Best-fitting redshift z: 0.03375916874846463

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -314       288
chi2/DOF: 1.238
method = capfi

 Best Fit:       Vel     sigma
 comp.  0:      -298       291
chi2/DOF: 1.099
method = capfit; Jac calls: 6; Func calls: 22; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
5 8
Vel :-297.821013.
Disp:290.650448.
Best-fitting redshift z: 0.03397180619473139

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -331       300
chi2/DOF: 1.128
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
5 9
Vel :-330.825458.
Disp:300.078178.
Best-fitting redshift z: 0.03385786203175534

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -322       284
chi2/DOF: 1.009
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  17 / 1273
5 10
Vel :-322.138023.
Disp:283.786494.
Best-fitting redshift z: 0.033887854431879694

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -314       318
chi2/DOF: 1.018
method = c

...

 Best Fit:       Vel     sigma
 comp.  0:      -309       289
chi2/DOF: 1.392
method = capfit; Jac calls: 7; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
14 17
Vel :-308.859769.
Disp:288.552553.
Best-fitting redshift z: 0.03393369611965191

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -342       259
chi2/DOF: 1.513
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  11 / 1273
14 18
Vel :-342.444302.
Disp:258.579569.
Best-fitting redshift z: 0.03381774926820902

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -320       266
chi2/DOF: 1.235
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
14 19
Vel :-320.373371.
Disp:265.923885.
Best-fitting redshift z: 0.033893946696067934

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -330       290
chi2/DOF: 1.613
metho

 Best Fit:       Vel     sigma
 comp.  0:      -372       276
chi2/DOF: 1.181
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
16 4
Vel :-372.055803.
Disp:276.403492.
Best-fitting redshift z: 0.03371551886549229

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -325       310
chi2/DOF: 1.379
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  18 / 1273
16 5
Vel :-325.051718.
Disp:309.950964.
Best-fitting redshift z: 0.03387779522411227

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -286       268
chi2/DOF: 1.107
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
16 6
Vel :-286.420844.
Disp:267.786101.
Best-fitting redshift z: 0.03401116400443782

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -330       313
chi2/DOF: 1.121
method = 

 Best Fit:       Vel     sigma
 comp.  0:      -283       256
chi2/DOF: 1.149
method = capfit; Jac calls: 6; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
17 11
Vel :-283.039879.
Disp:256.382650.
Best-fitting redshift z: 0.03402283641003945

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -335       320
chi2/DOF: 1.27
method = capfit; Jac calls: 6; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
17 12
Vel :-334.623348.
Disp:320.429233.
Best-fitting redshift z: 0.033844750239541366

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -346       274
chi2/DOF: 1.401
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
17 13
Vel :-346.092070.
Disp:274.413218.
Best-fitting redshift z: 0.0338051557588801

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -321       322
chi2/DOF: 1.476
method =

 Best Fit:       Vel     sigma
 comp.  0:      -214       332
chi2/DOF: 1.422
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
18 18
Vel :-213.866816.
Disp:331.809986.
Best-fitting redshift z: 0.034261648689146273

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -298       233
chi2/DOF: 1.365
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
18 19
Vel :-298.202159.
Disp:232.581260.
Best-fitting redshift z: 0.03397049032943178

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -305       230
chi2/DOF: 1.645
method = capfit; Jac calls: 7; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  20 / 1273
19 0
Vel :-305.039374.
Disp:230.250597.
Best-fitting redshift z: 0.03394688560699932

####################

1
 Best Fit:       Vel     sigma
 comp.  0:      -366       235
chi2/DOF: 1.468
method

<Figure size 432x288 with 0 Axes>

In [3]:
np.savetxt('Test2.txt',table, header= 'y_px x_px vel(km/s) disp(km/s) chi2/DOF 1-sigma', fmt='%i %i %.18e %.18e %.18e %.18e',delimiter=' ') 